In [13]:
import ollama
import torch
from transformers import AutoModelForImageClassification, AutoImageProcessor
from torchvision import transforms
from PIL import Image

In [14]:
model = 'llama3.2:1b'  # Set your model name here

In [15]:
if torch.cuda.is_available():
    print("CUDA is available. GPU is being used.")
else:
    print("CUDA is not available. Using CPU.")

CUDA is available. GPU is being used.


In [22]:
# Load the general food category model and processor
category_processor = AutoImageProcessor.from_pretrained("Kaludi/food-category-classification-v2.0")
category_model = AutoModelForImageClassification.from_pretrained("Kaludi/food-category-classification-v2.0")

# Load item-specific models
fruit_veg_model = AutoModelForImageClassification.from_pretrained("jazzmacedo/fruits-and-vegetables-detector-36")

# Define preprocessing transformation
preprocess = transforms.Compose([
    transforms.Resize(224),         # Resize to ensure the smaller edge is 512
    transforms.CenterCrop(224),     # Center crop to 512x512
    transforms.ToTensor(),          # Convert to tensor
])

# Mapping from category labels to specific item models
category_to_model = {
    "Fruit": fruit_veg_model,
    "Vegetable": fruit_veg_model
}

# Confidence threshold for item prediction
CONFIDENCE_THRESHOLD = 0.5

def classify_images(*images) -> list:
    """
    Accepts multiple PIL images, applies preprocessing, and returns a list of tuples with category and item labels.

    Parameters:
    images (PIL.Image.Image): Multiple PIL images provided as separate arguments.

    Returns:
    list: A list of tuples containing (category, item) for each image. If confidence is low or no model is available, item is None.
    """
    # Preprocess each image and create batch tensor
    preprocessed_images = [preprocess(image).unsqueeze(0) for image in images]
    batch = torch.cat(preprocessed_images, dim=0)  # Combine into a single batch tensor

    results = []
    
    # Run category model on the batch
    with torch.no_grad():
        category_outputs = category_model(batch)  # Get category predictions

    # Process each image's predictions for category and item
    for idx, category_logits in enumerate(category_outputs.logits):
        # Determine the category for each image
        category_idx = torch.argmax(category_logits).item()
        category_label = category_model.config.id2label[category_idx]

        # Check if there is a specific item model for the detected category
        item_model = category_to_model.get(category_label)

        # If no model is available, set item to None
        if item_model is None:
            results.append((category_label, None))
            continue

        # Classify the specific item if a model is available
        with torch.no_grad():
            item_logits = item_model(batch[idx].unsqueeze(0))  # Pass single image through item model
            item_probs = torch.softmax(item_logits.logits, dim=1)  # Get probability distribution
            item_confidence, item_idx = torch.max(item_probs, dim=1)
            item_confidence = item_confidence.item()
            item_idx = item_idx.item()
            item_label = item_model.config.id2label[item_idx]

        # Check if confidence meets threshold
        if item_confidence >= CONFIDENCE_THRESHOLD:
            results.append((category_label, item_label))
        else:
            results.append((category_label, None))
    
    return results

('Fruit', 'apple')
('Fruit', None)
('Meat', None)


In [23]:
import ollama
from PIL import Image

# Sample list of classified images
pil_images = [Image.open("../imgs/apple.jpg"), Image.open("../imgs/durian.jpg"), Image.open("../imgs/venison.jpg")]

# Call classify_images to get categories and item labels
classified_items = classify_images(*pil_images)

# Prepare ingredients list based on model output
ingredients = []
for category, item in classified_items:
    if item:
        ingredients.append(item)  # Use specific item if available
    else:
        ingredients.append(category)  # Use category as fallback if item is None

# Convert ingredients list into a prompt-friendly string
ingredient_list = ", ".join(ingredients)

# Generate recipe with Ollama based on available ingredients
response = ollama.chat(model=model, messages=[
    {
        'role': "user",
        'content': f"Create a unique recipe using the following ingredients: {ingredient_list}."
    },
])

print(response['message']['content'])

What an interesting combination. Here's a unique recipe that combines apples with meat:

Pan-Seared Applewood-Smoked Bacon-Wrapped Chicken Shawarma Wraps with Caramelized Onion Apple Chutney and Crispy Fried Apple Slices.

Ingredients:

For the chicken:

* 4 boneless, skinless chicken breasts
* 6 slices of applewood-smoked bacon, cut into thin strips
* 2 tbsp olive oil
* 1 tsp ground cumin
* Salt and pepper to taste

For the caramelized onion apple chutney:

* 1 large onion, thinly sliced
* 2 apples, diced (Granny Smith or Honeycrisp work well)
* 1/4 cup brown sugar
* 2 tbsp apple cider vinegar
* 1 tsp ground cinnamon
* Salt and pepper to taste

For the crispy fried apple slices:

* 2-3 apples, peeled and sliced into thin wedges
* 1 cup all-purpose flour
* 1 tsp paprika
* 1/2 tsp cayenne pepper (optional)
* 1/4 cup panko breadcrumbs
* Vegetable oil for frying

For the shawarma wraps:

* 4 large flour tortillas
* Salt and pepper to taste
* Optional toppings: pickled turnips, sumac, crum